# ephrin_binding
Analyze receptor selection data using soluble Ephrin-B2 or -B3 NOTE NEED TO FIX
- Written by Brendan Larsen

In [ ]:
# this cell is tagged as parameters for `papermill` parameterization
# input configs
altair_config = None
nipah_config = None

# input files
#entropy_file = None
#func_scores_E2_file = None
binding_E2_file = None
#func_scores_E3_file = None
binding_E3_file = None

# output files
#filtered_E2_binding_data = None
#filtered_E3_binding_data = None
#filtered_E2_binding_low_effect = None
#filtered_E3_binding_low_effect = None

# output images
entry_binding_combined_corr_plot = None
entry_binding_combined_corr_plot_agg = None
E2_E3_correlation = None
E2_E3_correlation_site = None
combined_E2_E3_site_corr = None
binding_by_site_plot = None
entry_binding_corr_heatmap = None
binding_corr_heatmap = None
binding_region_boxplot_plot = None
binding_region_bubble_plot = None
max_binding_in_stalk = None
max_binding_in_contact = None

In [ ]:
import math
import os
import re
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats
import yaml

In [ ]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

if (
    os.getcwd()
    == "/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/"
):
    pass
    print("Already in correct directory")
else:
    os.chdir(
        "/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/"
    )
    print("Setup in correct directory")

### hard paths for running in interactive mode

In [ ]:
if nipah_config is None:
    ##hard paths in case don't want to run with snakemake
    print("loading hard paths")
    altair_config = "data/custom_analyses_data/theme.py"
    nipah_config = "nipah_config.yaml"
    #entropy_file = "results/entropy/entropy.csv"

    # input files
    #func_scores_E2_file = "results/func_effects/averages/CHO_EFNB2_low_func_effects.csv"
    binding_E2_file = "results/filtered_data/binding/e2_binding_filtered.csv"

    #func_scores_E3_file = "results/func_effects/averages/CHO_EFNB3_low_func_effects.csv"
    binding_E3_file = "results/filtered_data/binding/e3_binding_filtered.csv"

    #filtered_E2_binding_data = "results/filtered_data/E2_binding_filtered.csv"
    #filtered_E3_binding_data = "results/filtered_data/E3_binding_filtered.csv"
    #filtered_E2_binding_low_effect = "results/filtered_data/E2_binding_low_effect_filter.csv"
    #filtered_E3_binding_low_effect = "results/filtered_data/E3_binding_low_effect_filter.csv"

### Run config files to setup altair theme and config variables

In [ ]:
if altair_config:
    with open(altair_config, "r") as file:
        exec(file.read())

with open(nipah_config) as f:
    config = yaml.safe_load(f)

### Import the binding and entry data for EFNB2 and EFNB3

In [ ]:
# import binding and entry data
df_E2_filter = pd.read_csv(binding_E2_file)
#e2_func = pd.read_csv(func_scores_E2_file)
df_E3_filter = pd.read_csv(binding_E3_file)
#e3_func = pd.read_csv(func_scores_E3_file)

### Filter the data and save 

In [ ]:
#def merge_func_binding_dfs(func, binding, name):
#    # Merge the 'binding' and 'func' DataFrames on specified columns with outer join.
#    # This combines data based on 'site', 'mutant', and 'wildtype' columns.
#    # Suffixes are added to columns with identical names in both DataFrames for differentiation.
#    # The result is rounded to three decimal places for neatness.
#    df_int = pd.merge(
#        binding,
#        func,
#        on=["site", "mutant", "wildtype"],
#        suffixes=["_binding", "_cell_entry"],
#        validate="one_to_one",
#        how="outer",
#    ).round(3)
#
#    # Rename specific columns for clarity.
#    df = df_int.rename(
#        columns={
#            "Ephrin binding_mean": "binding_mean",
#            "Ephrin binding_std": "binding_std",
#            "Ephrin binding_median": "binding_median",
#        }
#    )
#
#    # Select and reorder a subset of columns for the final DataFrame.
#    df = df[
#        [
#            "site",
#            "wildtype",
#            "mutant",
#            "binding_mean",
#            "binding_std",
#            "times_seen_binding",
#            "effect",
#            "effect_std",
#            "times_seen_cell_entry",
#            "frac_models",
#        ]
#    ]
#
#    # Define a function to filter the DataFrame based on several criteria.
#    def filter_binding_data(df):
#        # Apply multiple conditions to filter the data for relevant entries.
#        df_filter = df[
#            (df["mutant"] != "*")
#            & (df["mutant"] != "-")
#            & (df["site"] != 603)
#            &
#            # Conditions related to cell entry effects and observations.
#            (df["effect"] >= config["min_func_effect_for_binding"])
#            & (df["times_seen_cell_entry"] >= config["func_times_seen_cutoff"])
#            & (df["effect_std"] <= config["func_std_cutoff"])
#            &
#            # Conditions related to binding observations and variability.
#            (df["times_seen_binding"] >= config["min_times_seen_binding"])
#            & (df["binding_std"] <= config["max_binding_std"])
#            & (df["frac_models"] >= config["frac_models"])
#        ]
#        return df_filter
#
#    # Filter the DataFrame using the defined function.
#    df_filter = filter_binding_data(df)
#
#    # Define a function to identify low effect mutants for further analysis.
#    def store_filtered_info(df):
#        df_low_filter = df[
#            (df["mutant"] != "*")
#            & (df["mutant"] != "-")
#            & (df["site"] != 603)
#            & (df["effect"] < config["min_func_effect_for_binding"])
#            & (df["times_seen_cell_entry"] >= config["func_times_seen_cutoff"])
#            & (df["effect_std"] <= config["func_std_cutoff"])
#        ]
#        return df_low_filter
#
#    # Filter the DataFrame to identify low effect mutants.
#    df_low_effect_filter = store_filtered_info(df)
#
#    # Save the filtered data to CSV files, differentiating between EFNB2 and others.
#    if name == "EFNB2":
#        print(name)  # Print the name for confirmation.
#        df_filter.to_csv(filtered_E2_binding_data, index=False)
#        df_low_effect_filter.to_csv(filtered_E2_binding_low_effect, index=False)
#    else:
#        df_filter.to_csv(filtered_E3_binding_data, index=False)
#        df_low_effect_filter.to_csv(filtered_E3_binding_low_effect, index=False)
#
#    # Return the filtered DataFrames for further use.
#    return df_filter, df_low_effect_filter
#
## Use the defined function to filter data for EFNB2 and EFNB3.
#df_E2_filter, df_E2_filter_missing = merge_func_binding_dfs(e2_func, e2, "EFNB2")
#df_E3_filter, df_E3_filter_missing = merge_func_binding_dfs(e3_func, e3, "EFNB3")
#
## Merge the filtered EFNB2 and EFNB3 DataFrames for combined analysis.
df_binding_effect_merge = pd.merge(
    df_E2_filter,
    df_E3_filter,
    on=["site", "wildtype", "mutant"],
    suffixes=["_E2", "_E3"],
    how="outer",
)

## Display descriptive statistics of the merged DataFrame.
#display(df_binding_effect_merge.describe().round(3))
#
## Add a 'selection' column to distinguish between EFNB2 and EFNB3 data.
df_E2_filter["selection"] = "EFNB2"
df_E3_filter["selection"] = "EFNB3"
#
## Concatenate the DataFrames for plotting or further analysis.
df_binding_effect_concat = pd.concat([df_E2_filter, df_E3_filter])

In [ ]:
#def what_got_filtered(df,name):
#    print(f' Filtering stats for {name}:\n')
#    print(df.shape[0])
#    print(df.dropna().shape[0])
#    display(df)
#    tmp_df = df[df['effect'] < config['min_func_effect_for_binding']]
#    print(tmp_df.shape[0])
#
#
#what_got_filtered(df_E2_filter_missing,'EFNB2')
#what_got_filtered(df_E3_filter_missing,'EFNB2')

In [ ]:
# What are the top 5 highest and lowest binding mutants for EFNB2 and EFNB3?
def find_highest_lowest(df, name):
    #df = df[df['site'].isin(config['contact_sites'])]
    print(f"We are analyzing {name}\n")
    print(f"The total number of mutants was: {df.shape[0]}\n")
    tmp_df = df.sort_values(by="binding_mean")
    print("These are the lowest binding mutants detected:")
    display(tmp_df.head(5))

    tmp_df_high = df.sort_values(by="binding_mean", ascending=False)
    print("\nThese are the highest binding mutants detected:\n")
    display(tmp_df_high.head(5))

    # What about mutants with positive entry scores?
    tmp_df = df[df["effect"] > 0].sort_values(by="binding_mean")
    print("These are the lowest binding mutants detected with positive entry scores:")
    display(tmp_df.head(5))

    tmp_df_high = df[df["effect"] > 0].sort_values(by="binding_mean", ascending=False)
    print(
        "\nThese are the highest binding mutants detected with positive entry scores:\n"
    )
    display(tmp_df_high.head(5))

    mean_df = df.groupby('site')['binding_mean'].sum().reset_index()
    display(mean_df.sort_values(by='binding_mean',ascending=False).head(10))


find_highest_lowest(df_E2_filter, "EFNB2")
find_highest_lowest(df_E3_filter, "EFNB3")

### Find the top overlapping binders for both EFNB2 and EFNB3

In [ ]:
def find_good_binding_for_both(df):
    e2_good = df.sort_values(by='binding_mean_E2',ascending=False)
    e3_good = df.sort_values(by='binding_mean_E3',ascending=False)
    e2_good = e2_good.head(20)
    e3_good = e3_good.head(20)
    combo = pd.merge(e2_good,e3_good,on=['site','wildtype','mutant'],how='inner')
    display(combo)

find_good_binding_for_both(df_binding_effect_merge)

In [ ]:
# Compare E2 and E3 binders
def find_highest_lowest(df):
    df["binding_diff"] = (df["binding_mean_E2"] - df["binding_mean_E3"]).abs()
    print(
        "These are the mutants with the biggest difference between EFNB2 and EFNB3:\n"
    )
    display(df.sort_values(by="binding_diff", ascending=False).head(10))

    # calculate aggregate differences
    agg_df = (
        df.groupby("site")[["binding_mean_E2", "binding_mean_E3", "binding_diff"]]
        .mean()
        .reset_index()
    )
    print("These are the sites with the biggest difference between EFNB2 and EFNB3:\n")
    display(agg_df.sort_values(by="binding_diff", ascending=False).head(5))


find_highest_lowest(df_binding_effect_merge)
# find_highest_lowest(df_E3_filter,'EFNB3')

### Make plots showing correlation between binding and entry for EFNB2 and EFNB3

In [ ]:
def plot_corr_binding_entry_updated(df, flag):
    # Define interactive selectors for variant selection.
    # 'variant_selector' is for selecting individual variants based on 'site' and 'mutant'.
    variant_selector = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site", "mutant"], value=0
    )
    # 'variant_selector_agg' is for aggregated selection based on 'site' only.
    variant_selector_agg = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site"], value=0
    )

    # Initialize an empty list to store charts for each unique selection in 'df'.
    empty_chart = []
    
    # Loop through each unique cell selection in the DataFrame.
    for cell in list(df["selection"].unique()):
        # Filter DataFrame for the current selection.
        tmp_df = df[df["selection"] == cell]
        
        # Check if aggregation flag is True to aggregate data.
        if flag == True:
            # Aggregate data by 'site', summing 'binding_median' and 'effect', then reset index.
            agg_df = (
                tmp_df.groupby("site")[["binding_mean", "effect"]].sum().reset_index()
            )
            # Create a chart using aggregated data with specified encodings.
            chart = (
                alt.Chart(agg_df)
                .mark_point(stroke="black", filled=True)  # Use filled points with black stroke.
                .encode(
                    x=alt.X(
                        "effect",
                        title=f"Summed {cell} Cell Entry",
                        axis=alt.Axis(grid=True),
                    ),
                    y=alt.Y(
                        "binding_mean",
                        title=f"Summed {cell} Binding",
                        axis=alt.Axis(grid=True),
                    ),
                    # Dynamic opacity, size, strokeWidth, and color based on 'variant_selector_agg'.
                    opacity=alt.condition(
                        variant_selector_agg, alt.value(1), alt.value(0.2)
                    ),
                    size=alt.condition(
                        variant_selector_agg, alt.value(100), alt.value(50)
                    ),
                    strokeWidth=alt.condition(
                        variant_selector_agg, alt.value(1), alt.value(0)
                    ),
                    color=alt.condition(
                        variant_selector_agg, alt.value("orange"), alt.value("black")
                    ),
                    tooltip=["site", "binding_mean", "effect"],
                )
                .add_params(variant_selector_agg)
            )
            # Add the chart to the list.
            empty_chart.append(chart)

        else:
            # Create a chart for individual data points with specified encodings.
            chart = (
                alt.Chart(tmp_df)
                .mark_point(stroke="black", filled=True)
                .encode(
                    x=alt.X(
                        "effect", title=f"{cell} Cell Entry", axis=alt.Axis(grid=True)
                    ),
                    y=alt.Y(
                        "binding_mean",
                        title=f"{cell} Binding",
                        axis=alt.Axis(grid=True),
                    ),
                    # Dynamic opacity, size, strokeWidth, and color based on 'variant_selector'.
                    opacity=alt.condition(
                        variant_selector, alt.value(1), alt.value(0.1)
                    ),
                    size=alt.condition(variant_selector, alt.value(50), alt.value(20)),
                    strokeWidth=alt.condition(
                        variant_selector, alt.value(1), alt.value(0)
                    ),
                    color=alt.condition(
                        variant_selector, alt.value("orange"), alt.value("black")
                    ),
                    tooltip=[
                        "site",
                        "wildtype",
                        "mutant",
                        "binding_mean",
                        "times_seen_binding",
                        "effect",
                    ],
                )
                .add_params(variant_selector)
            )
            # Add the chart to the list.
            empty_chart.append(chart)

    # Combine all charts horizontally with a title.
    combined_chart = alt.hconcat(
        *empty_chart, title=alt.Title("Correlation between binding and entry")
    )
    
    # Return the combined chart for display or further use.
    return combined_chart

# Generate and display plots for non-aggregated data.
entry_binding_corr_plot = plot_corr_binding_entry_updated(df_binding_effect_concat, False)
entry_binding_corr_plot.display()

# Save the plot 
if entry_binding_combined_corr_plot is not None:
    entry_binding_corr_plot.save(entry_binding_combined_corr_plot)

# Repeat for aggregated data.
entry_binding_corr_plot_agg = plot_corr_binding_entry_updated(df_binding_effect_concat, True)
entry_binding_corr_plot_agg.display()

# Save the aggregated plot
if entry_binding_combined_corr_plot is not None:
    entry_binding_corr_plot_agg.save(entry_binding_combined_corr_plot_agg)

In [ ]:
def plot_corr_binding_entry_updated(df, flag):
    # Define interactive selectors for variant selection.
    # 'variant_selector' is for selecting individual variants based on 'site' and 'mutant'.
    variant_selector = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site", "mutant"], value=0
    )
    # 'variant_selector_agg' is for aggregated selection based on 'site' only.
    variant_selector_agg = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site"], value=0
    )

    # Initialize an empty list to store charts for each unique selection in 'df'.
    empty_chart = []
    
    # Loop through each unique cell selection in the DataFrame.
    for cell in list(df["selection"].unique()):
        # Filter DataFrame for the current selection.
        tmp_df = df[df["selection"] == cell]
        
        # Check if aggregation flag is True to aggregate data.
        if flag == True:
            # Aggregate data by 'site', summing 'binding_median' and 'effect', then reset index.
            agg_df = (
                tmp_df.groupby("site")[["binding_mean", "effect"]].sum().reset_index()
            )
            # Create a chart using aggregated data with specified encodings.
            chart = (
                alt.Chart(agg_df)
                .mark_point(stroke="black", filled=True)  # Use filled points with black stroke.
                .encode(
                    x=alt.X(
                        "effect",
                        title=f"Summed {cell} Cell Entry",
                        axis=alt.Axis(grid=True),
                    ),
                    y=alt.Y(
                        "binding_mean",
                        title=f"Summed {cell} Binding",
                        axis=alt.Axis(grid=True),
                    ),
                    # Dynamic opacity, size, strokeWidth, and color based on 'variant_selector_agg'.
                    opacity=alt.condition(
                        variant_selector_agg, alt.value(1), alt.value(0.2)
                    ),
                    size=alt.condition(
                        variant_selector_agg, alt.value(100), alt.value(50)
                    ),
                    strokeWidth=alt.condition(
                        variant_selector_agg, alt.value(1), alt.value(0)
                    ),
                    color=alt.condition(
                        variant_selector_agg, alt.value("orange"), alt.value("black")
                    ),
                    tooltip=["site", "binding_mean", "effect"],
                )
                .add_params(variant_selector_agg)
            )
            # Add the chart to the list.
            empty_chart.append(chart)

        else:
            # Create a chart for individual data points with specified encodings.
            chart = (
                alt.Chart(tmp_df)
                .mark_point(stroke="black", filled=True)
                .encode(
                    x=alt.X(
                        "effect", title=f"{cell} Cell Entry", axis=alt.Axis(grid=True)
                    ),
                    y=alt.Y(
                        "binding_mean",
                        title=f"{cell} Binding",
                        axis=alt.Axis(grid=True),
                    ),
                    # Dynamic opacity, size, strokeWidth, and color based on 'variant_selector'.
                    opacity=alt.condition(
                        variant_selector, alt.value(1), alt.value(0.1)
                    ),
                    size=alt.condition(variant_selector, alt.value(50), alt.value(20)),
                    strokeWidth=alt.condition(
                        variant_selector, alt.value(1), alt.value(0)
                    ),
                    color=alt.condition(
                        variant_selector, alt.value("orange"), alt.value("black")
                    ),
                    tooltip=[
                        "site",
                        "wildtype",
                        "mutant",
                        "binding_mean",
                        "times_seen_binding",
                        "effect",
                    ],
                )
                .add_params(variant_selector)
            )
            # Add the chart to the list.
            empty_chart.append(chart)

    # Combine all charts horizontally with a title.
    combined_chart = alt.hconcat(
        *empty_chart, title=alt.Title("Correlation between binding and entry")
    )
    
    # Return the combined chart for display or further use.
    return combined_chart

# Generate and display plots for non-aggregated data.
entry_binding_corr_plot = plot_corr_binding_entry_updated(df_binding_effect_concat, False)
entry_binding_corr_plot.display()

### Same plot as above, but slightly different format

In [ ]:
def plot_entry_binding_corr_heatmap(df):
    empty_chart = []

    for cell in list(df["selection"].unique()):
        tmp_df = df[df["selection"] == cell]
        chart = (
            alt.Chart(tmp_df, title=f"{cell}")
            .mark_rect()
            .encode(
                x=alt.X(
                    "effect", title="Cell Entry", axis=alt.Axis(values=[-2, -1, 0, 1])
                ).bin(maxbins=60),
                y=alt.Y(
                    "binding_mean",
                    title="Binding",
                    axis=alt.Axis(values=[-4, -2, 0, 2]),
                ).bin(maxbins=60),
                color=alt.Color("count()", title="Count").scale(scheme="greenblue"),
            )
        )
        empty_chart.append(chart)

    combined_chart = alt.hconcat(
        *empty_chart, title=alt.Title("Correlation between binding and entry")
    ).resolve_scale(y="shared", x="shared", color="shared")
    return combined_chart


entry_binding_corr_heat = plot_entry_binding_corr_heatmap(df_binding_effect_concat)
entry_binding_corr_heat.display()
if entry_binding_combined_corr_plot is not None:
    entry_binding_corr_heat.save(entry_binding_corr_heatmap)

### Calculate some stats on binding

In [ ]:
def overall_stats(df, effect, name):
    # Now group sites and find sites where all mutants are deleterious
    filtered_df = df.groupby("site").filter(lambda group: (group[effect] < -0.25).all())
    # Which sites are these?
    unique = filtered_df["site"].unique()
    # Convert unique to a Pandas Series
    unique_series = pd.Series(unique)

    # Find the common elements that are also contact sites
    unique_contact_bool = unique_series.isin(config["contact_sites"])
    # Filter and get the common elements
    common_elements = unique_series[unique_contact_bool]

    print(f"The dataset we are analyzing is: {name}\n")

    # Print the common elements
    print(
        f"Here are the contact sites that only have negative binding scores: {list(common_elements)}\n"
    )

    print(f"There are {len(unique)} sites with all negative binding score mutants\n")
    print(
        f"These are the sites with all negative binding score mutants: {list(unique)}\n"
    )

    # Now find sites with low and high binding (median)
    median_df = (
        df.groupby("site")["binding_mean"]
        .max()
        .reset_index()
        .sort_values(by="binding_mean", ascending=False)
    )
    print("These are the sites with the highest binding mutants:\n")
    display(median_df.head(5))

    # Now calculate mutant number
    total_mutants = df.shape[0]

    mutants_above_cutoff_tolerated = df[df["effect"] > 0]
    mutants_above_cutoff_tolerated = mutants_above_cutoff_tolerated[
        ["site", "effect", "binding_mean", "wildtype", "mutant"]
    ]

    total_sites = df["site"].unique().shape[0]

    print(f"The total number of sites are: {total_sites}")


overall_stats(df_E2_filter, "binding_mean", "EFNB2")
overall_stats(df_E3_filter, "binding_mean", "EFNB3")

### Find sites with opposite effects on binding

In [ ]:
# find sites that are different
def find_biggest_differences(df):
    efnb2_good_efnb3_bad = df[
        (df["binding_mean_E2"] > 0.5) & (df["binding_mean_E3"] < -0.5)
    ].sort_values(by="binding_mean_E2", ascending=False)
    display(efnb2_good_efnb3_bad)

    efnb2_bad_efnb3_good = df[
        (df["binding_mean_E2"] < -0.5) & (df["binding_mean_E3"] > 0.5)
    ].sort_values(by="binding_mean_E3", ascending=False)
    display(efnb2_bad_efnb3_good)


find_biggest_differences(df_binding_effect_merge)

### Find correlations between EFNB2 and EFNB3 binding

In [ ]:
def plot_entry_binding_corr(df):
    chart = (
        alt.Chart(df, title="Correlation Between Mutant Binding Scores")
        .mark_rect()
        .encode(
            x=alt.X(
                "binding_mean_E2",
                title="EFNB2 binding",
                axis=alt.Axis(values=[-5, 0, 2]),
            ).bin(maxbins=40),
            y=alt.Y(
                "binding_mean_E3",
                title="EFNB3 binding",
                axis=alt.Axis(values=[-2, 0, 2]),
            ).bin(maxbins=40),
            color=alt.Color("count()", title="Count").scale(scheme="greenblue"),
        )
    )
    return chart


entry_binding_corr_heatmap_1 = plot_entry_binding_corr(df_binding_effect_merge)
entry_binding_corr_heatmap_1.display()
if entry_binding_combined_corr_plot is not None:
    entry_binding_corr_heatmap_1.save(binding_corr_heatmap)

In [ ]:
def plot_affinity_solid(df):
    df = df.dropna()
    # calculate r value
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
        df["binding_mean_E2"], df["binding_mean_E3"]
    )
    r_value = float(r_value)
    # make chart
    chart = (
        alt.Chart(
            df,
            title=alt.Title(
                "Correlation between Mutant Binding Scores", subtitle=f"r={r_value:.2f}"
            ),
        )
        .mark_point(color="black", size=30, opacity=0.2, filled=True)
        .encode(
            x=alt.X("binding_mean_E2", title=("EFNB2 Binding")),
            y=alt.Y("binding_mean_E3", title=("EFNB3 Binding")),
            tooltip=[
                "site",
                "wildtype",
                "mutant",
                "binding_mean_E2",
                "binding_mean_E3",
                "effect_E2",
                "effect_E3",
            ],
        )
    )
    highlight = chart.transform_filter(
        (alt.datum.site == 458) & (alt.datum.mutant == 'Y') | # Example condition
        (alt.datum.site == 507) & (alt.datum.mutant == 'I') #|
        #(alt.datum.site == 589) & (alt.datum.mutant == 'G') |
        #(alt.datum.site == 580) & (alt.datum.mutant == 'S') | 
        #(alt.datum.site == 492) & (alt.datum.mutant == 'L') |
        #(alt.datum.site == 492) & (alt.datum.mutant == 'R') |
        #(alt.datum.site == 566) & (alt.datum.mutant == 'C') |
        #(alt.datum.site == 211) & (alt.datum.mutant == 'F') |
        #(alt.datum.site == 553) & (alt.datum.mutant == 'W') |
        #(alt.datum.site == 546) & (alt.datum.mutant == 'H') |
        #(alt.datum.site == 555) & (alt.datum.mutant == 'K') 

        


    ).mark_point(
        color="black", size=40, opacity=1, filled=True, stroke='orange', strokeWidth=1
    )
    min = int(df["binding_mean_E2"].min())
    max = int(df["binding_mean_E3"].max())
    text = (
        alt.Chart({"values": [{"x": min, "y": max, "text": f"r = {r_value:.2f}"}]})
        .mark_text(align="left", baseline="top", dx=-10, dy=-20)
        .encode(x=alt.X("x:Q"), y=alt.Y("y:Q"), text="text:N")
    )
    chart_and_text = chart + highlight
    return chart_and_text


E2_E3_corr = plot_affinity_solid(df_binding_effect_merge)
E2_E3_corr.display()
if entry_binding_combined_corr_plot is not None:
    E2_E3_corr.save(E2_E3_correlation)

### Plot correlations between summary statistics for each site

In [ ]:
def plot_affinity_solid_mean(df):
    df = df.dropna()
    means = (
        df.groupby("site")
        .agg(
            {
                "effect_E2": "mean",
                "effect_E3": "mean",
                "binding_mean_E2": "mean",
                "binding_mean_E3": "mean",
                "wildtype": "first",
            }
        )
        .reset_index()
    )
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
        means["binding_mean_E2"], means["binding_mean_E3"]
    )
    r_value = float(r_value)
    chart = (
        alt.Chart(
            means,
            title=alt.Title(
                "Correlation between Aggregate Mutant Binding Scores",
                subtitle=f"r={r_value:.2f}",
            ),
        )
        .mark_point(size=50, opacity=0.3)
        .encode(
            x=alt.X(
                "binding_mean_E2",
                title=("Avg EFNB2 Binding"),
                axis=alt.Axis(tickCount=3),
            ),
            y=alt.Y(
                "binding_mean_E3",
                title=("Avg EFNB3 Binding"),
                axis=alt.Axis(tickCount=3),
            ),
            tooltip=[
                "site",
                "wildtype",
                "binding_mean_E2",
                "binding_mean_E3",
                "effect_E2",
                "effect_E3",
            ],
        )
    )
    text = (
        alt.Chart({"values": [{"x": -3.5, "y": 0.5, "text": f"r = {r_value:.2f}"}]})
        .mark_text(align="left", baseline="top", dx=0, dy=0)
        .encode(x=alt.X("x:Q"), y=alt.Y("y:Q"), text="text:N")
    )
    chart_and_text = chart
    return chart_and_text


E2_E3_site_corr = plot_affinity_solid_mean(df_binding_effect_merge)
E2_E3_site_corr.display()
if entry_binding_combined_corr_plot is not None:
    E2_E3_site_corr.save(E2_E3_correlation_site)

if entry_binding_combined_corr_plot is not None:
    (E2_E3_corr | E2_E3_site_corr).save(combined_E2_E3_site_corr)

### Make plot showing binding by site (median)

In [ ]:
def plot_affinity_by_site_median(df):
    variant_selector = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site"], value=0
    )
    empty_charts = []
    for selection in ["binding_mean_E2", "binding_mean_E3"]:
        if selection == "binding_mean_E2":
            name = "EFNB2 Binding"
        else:
            name = "EFNB3 Binding"
        mean = df.groupby("site")[selection].max().reset_index()
        mean = mean[mean[selection] >= 0]
        chart = (
            alt.Chart(mean)
            .mark_point(stroke="black", filled=True, size=50)
            .encode(
                x=alt.X(
                    "site",
                    title=("Site"),
                    axis=alt.Axis(grid=True, tickCount=4),
                    scale=alt.Scale(domain=[70, 602]),
                ),
                y=alt.Y(selection, title=(name), axis=alt.Axis(grid=True, tickCount=3)),
                tooltip=["site"],
                color=alt.condition(
                    variant_selector, alt.value("orange"), alt.value("black")
                ),
                opacity=alt.condition(variant_selector, alt.value(1), alt.value(0.5)),
                strokeWidth=alt.condition(variant_selector, alt.value(1), alt.value(0)),
            )
            .properties(height=150, width=500)
            .add_params(variant_selector)
        )
        empty_charts.append(chart)
    combined_chart = alt.vconcat(*empty_charts, spacing=1, title="Max Binding by Site")
    return combined_chart


binding_by_site = plot_affinity_by_site_median(df_binding_effect_merge)
binding_by_site.display()
if entry_binding_combined_corr_plot is not None:
    binding_by_site.save(binding_by_site_plot)

In [ ]:
def plot_affinity_by_contact_site(df, sites_to_show, title_text):
    variant_selector = alt.selection_point(
        on="mouseover", nearest=True, empty=False, fields=["site"], value=0
    )
    empty_charts = []

    contact_df = df[df["site"].isin(sites_to_show)]
    sites = list(contact_df["site"].unique())

    for selection in df["selection"].unique():
        tmp_df = contact_df[contact_df["selection"] == selection]
        mean = tmp_df.groupby("site")["binding_mean"].max().reset_index()

        chart = (
            alt.Chart(mean)
            .mark_point(size=100)
            .encode(
                x=alt.X(
                    "site:O",
                    sort=sites,
                    title=("Site"),
                    axis=alt.Axis(grid=True, labelAngle=-90),
                    scale=alt.Scale(domain=sites),
                ),
                y=alt.Y(
                    "binding_mean", title=(f"{selection}"), axis=alt.Axis(grid=True)
                ),
                tooltip=["site"],
                color=alt.condition(
                    variant_selector, alt.value("orange"), alt.value("black")
                ),
                strokeWidth=alt.condition(variant_selector, alt.value(2), alt.value(0)),
            )
            .add_params(variant_selector)
        )
        empty_charts.append(chart)
    combined_chart = alt.vconcat(*empty_charts, spacing=1, title=title_text)
    return combined_chart


contact_binding_by_site = plot_affinity_by_contact_site(
    df_binding_effect_concat, config["contact_sites"], "Max Binding in Contact"
)
contact_binding_by_site.display()
if entry_binding_combined_corr_plot is not None:
    contact_binding_by_site.save(max_binding_in_contact)

contact_binding_by_site_stalk = plot_affinity_by_contact_site(
    df_binding_effect_concat, list(range(96, 147)), "Max Binding in Stalk"
)
contact_binding_by_site_stalk.display()
if entry_binding_combined_corr_plot is not None:
    contact_binding_by_site_stalk.save(max_binding_in_stalk)

### Make bubble plots for binding in different areas of receptor pocket

In [ ]:
def make_boxplot_binding_region(
    df, title
):  # Create a box plot using Altair for aggregated means
    barrel_ranges = {
        "Hydrophobic": config["hydrophobic"],
        "Salt Bridges": config["salt_bridges"],
        "Hydrogen Bonds": config["h_bond_total"],
        "Contact": config["contact_sites"],
        "Overall": list(range(71, 602)),
    }

    mean_df = df.groupby("site")[["binding_mean"]].median().reset_index()
    custom_order = [
        "Hydrophobic",
        "Salt Bridges",
        "Hydrogen Bonds",
        "Contact",
        "Overall",
    ]
    agg_means = []

    # For each barrel, filter the site_means dataframe to the sites belonging to that barrel and then store the means
    for barrel, sites in barrel_ranges.items():
        subset = mean_df[mean_df["site"].isin(sites)]
        for _, row in subset.iterrows():
            agg_means.append(
                {"barrel": barrel, "effect": row["binding_mean"], "site": row["site"]}
            )
        agg_means_df = pd.DataFrame(agg_means)
    chart = (
        alt.Chart(agg_means_df)
        .mark_point(size=50, opacity=0.4)
        .encode(
            x=alt.X(
                "barrel:O", sort=custom_order, title=None, axis=alt.Axis(labelAngle=-90)
            ),
            y=alt.Y(
                "effect",
                title=f"Median {title} Binding",
                axis=alt.Axis(grid=True, tickCount=4),
            ),
            xOffset="random:Q",
            tooltip=["barrel", "effect", "site"],
        )
        .transform_calculate(random="sqrt(-1*log(random()))*cos(2*PI*random())")
    )

    return chart.display()


make_boxplot_binding_region(df_E2_filter, "EFNB2")
make_boxplot_binding_region(df_E3_filter, "EFNB3")

#### make boxplot of binding scores by region

In [ ]:
def make_boxplot_binding_region(df):
    barrel_ranges = {
        "Stalk": list(range(96, 147)),
        "Neck": list(range(148, 165)),
        "Linker": list(range(166, 177)),
        "Head": list(range(178, 602)),
        "Receptor Contact": config["contact_sites"],
        "Total": list(range(71, 602)),
    }
    custom_order = ["Stalk", "Neck", "Linker", "Head", "Receptor Contact", "Total"]
    empty_charts = []
    for selection in df["selection"].unique():
        tmp_df = df[df["selection"] == selection]
        agg_means = []

        # For each barrel, filter the site_means dataframe to the sites belonging to that barrel and then store the means
        for barrel, sites in barrel_ranges.items():
            subset = tmp_df[tmp_df["site"].isin(sites)]
            for _, row in subset.iterrows():
                agg_means.append(
                    {
                        "region": barrel,
                        "binding_mean": row["binding_mean"],
                        "site": row["site"],
                    }
                )
            agg_means_df = pd.DataFrame(agg_means)

        chart = (
            alt.Chart(agg_means_df, title=f"{selection}")
            .mark_boxplot(color="darkgray", extent="min-max", opacity=1)
            .encode(
                x=alt.X(
                    "region:O",
                    sort=custom_order,
                    title="RBP Region",
                    axis=alt.Axis(labelAngle=-90),
                ),
                y=alt.Y(
                    "binding_mean",
                    title=f"Binding",
                    axis=alt.Axis(grid=True, tickCount=4),
                ),
                tooltip=["region", "binding_mean", "site"],
            )
            .properties(width=config["width"], height=config["height"])
        )
        empty_charts.append(chart)
    combined_effect_chart = alt.hconcat(*empty_charts).resolve_scale(
        y="shared", x="shared", color="independent"
    )
    return combined_effect_chart


entry_region_boxplot = make_boxplot_binding_region(df_binding_effect_concat)
entry_region_boxplot.display()
if entry_binding_combined_corr_plot is not None:
    entry_region_boxplot.save(binding_region_boxplot_plot)

In [ ]:
def make_bubble_binding_region(df):
    barrel_ranges = {
        "Stalk": list(range(96, 147)),
        "Neck": list(range(148, 165)),
        "Linker": list(range(166, 177)),
        "Head": list(range(178, 602)),
        "Receptor Contact": config["contact_sites"],
        #"Total": list(range(71, 602)),
    }
    custom_order = ["Stalk", "Neck", "Linker", "Head", "Receptor Contact"]
    empty_charts = []
    for selection in df["selection"].unique():
        tmp_df = df[df["selection"] == selection]
        agg_means = []

        # For each barrel, filter the site_means dataframe to the sites belonging to that barrel and then store the means
        for barrel, sites in barrel_ranges.items():
            subset = tmp_df[tmp_df["site"].isin(sites)]
            for _, row in subset.iterrows():
                agg_means.append(
                    {
                        "region": barrel,
                        "binding_mean": row["binding_mean"],
                        "site": row["site"],
                        "mutant": row["mutant"],
                    }
                )
            agg_means_df = pd.DataFrame(agg_means)

        variant_selector = alt.selection_point(
            on="mouseover", empty=False, fields=["site", "mutant"], value=1
        )

        chart = (
            alt.Chart(agg_means_df, title=f"{selection}")
            .mark_point(opacity=0.3, stroke="black")
            .encode(
                x=alt.X(
                    "region:O",
                    sort=custom_order,
                    title="RBP Region",
                    axis=alt.Axis(labelAngle=-90),
                ),
                y=alt.Y(
                    "binding_mean",
                    title=f"Binding",
                    axis=alt.Axis(grid=True, tickCount=4),
                ),
                xOffset="random:Q",
                tooltip=["region", "binding_mean", "site", "mutant"],
                color=alt.condition(
                    variant_selector, alt.value("orange"), alt.value("black")
                ),
                opacity=alt.condition(variant_selector, alt.value(1), alt.value(0.1)),
                strokeWidth=alt.condition(variant_selector, alt.value(2), alt.value(0)),
                size=alt.condition(variant_selector, alt.value(50), alt.value(15)),
            )
            .transform_calculate(random="sqrt(-1*log(random()))*cos(2*PI*random())")
            .properties(width=config["width"], height=config["height"])
        ).add_params(variant_selector)
        empty_charts.append(chart)
    combined_effect_chart = (
        alt.hconcat(*empty_charts)
        .resolve_scale(y="shared", x="shared", color="independent")
        .add_params(variant_selector)
    )
    return combined_effect_chart


entry_region_bubble = make_bubble_binding_region(df_binding_effect_concat)
entry_region_bubble.display()
if entry_binding_combined_corr_plot is not None:
    entry_region_bubble.save(binding_region_bubble_plot)